In [1]:
!pip3 install pypdf
!pip3 install langchain langchain_community
!pip3 install tiktoken


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
CHROMA_PATH = "chroma"

In [4]:
import os
import argparse
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [5]:
document_loader = PyPDFDirectoryLoader("/Users/rushabhrunwal/Downloads/SOFI_pdfs") # this  is the path to the folder containing the pdfs
documents = document_loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=80,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(documents)

In [7]:
def get_embedding_function():
  embeddings = OllamaEmbeddings(model="nomic-embed-text")
  return embeddings

In [8]:

def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id
        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id
    return chunks


In [ ]:
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
        print(f"Added documents: {len(new_chunks)}")


In [10]:
add_to_chroma(chunks)

/var/folders/m7/1q9b4n3x3_90cgd4x7g12db80000gn/T/ipykernel_77636/1436976703.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")
/var/folders/m7/1q9b4n3x3_90cgd4x7g12db80000gn/T/ipykernel_77636/3927975923.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_fu

Number of existing documents in DB: 0
Added documents: 3148


/var/folders/m7/1q9b4n3x3_90cgd4x7g12db80000gn/T/ipykernel_77636/3927975923.py:22: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [89]:
def load_Generative_model():
    model = Ollama(model="mistral")
    return model
model = load_Generative_model()

In [120]:
Submission_data = {}

In [121]:
def query_rag(query_text: str):
    PROMPT_TEMPLATE = """ Response the question based only on the following context:
    {context} \n\n\n\n
    Answer this question based on the above context: \n {question}
    """
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n\n _____________ \n\n\n".join([doc.page_content.rstrip() for doc, _score in results])
    contexts = "".join([doc.page_content.rstrip() for doc, _score in results])
    context_details = []
    for i, (doc, _score) in enumerate(results):
        context_details.append([doc.metadata['id'], _score])        
    
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    response_text = model.invoke(prompt)
    response_to_save = {"Response": response_text, "context_texts": context_text, "Sources": context_details}
    Submission_data[query_text] = response_to_save
    return response_text


In [122]:
res = query_rag("Explain increase prices impact on food security - comparing 2023 and 2024 reports")
print(res)

 The increases in global food prices throughout 2022 (as reported in the 2023 report) have significantly affected food security, as highlighted in the 2024 report. Here's a comparison between the two reports:

    - In the 2023 report, the average cost of a healthy diet globally was 3.56 PPP dollars per person per day. However, this figure rose to 3.96 PPP dollars in 2022 (as reported in the 2024 report), marking an increase of 11% between 2021 and 2022. This surge represents a strain on household budgets, particularly for those already struggling with food insecurity.
    - The World Food Import Bill was estimated to have reached an all-time high in 2022 (nearly USD $2 trillion), a 10% increase from the 2021 level. This significant rise in import costs affected countries heavily dependent on food imports, exacerbating their food security challenges.
    - World fertilizer prices also surged in 2022, primarily due to rising energy and natural gas prices. The global agricultural input i

In [123]:
res = query_rag("List major Food insecurity reason in 2024")
print(res)

1. Climatic factors: These can include extreme weather events, droughts, floods, and changes in temperature patterns that affect agricultural productivity.

2. Economic factors: Economic instability or recession can lead to job losses, reduced income, and increased food prices, making it difficult for people to afford nutritious food.

3. Social factors: These could include inequality, poverty, and lack of access to education and healthcare services which can all contribute to food insecurity.

4. Political factors: Political instability, conflicts, and poor governance can disrupt food production and distribution, leading to food insecurity.

5. Lack of affordability of a healthy diet: The report specifically mentions this as a factor contributing to food insecurity. Improper estimates of the cost and affordability of a healthy diet can contribute to this issue.

6. Urbanization: As populations become more urban, diets may shift towards less nutritious foods, which can lead to malnutri

In [124]:
res = query_rag("Explain malnutrition in war zones")
print(res)

 The provided context does not explicitly mention malnutrition in war zones. However, it does imply that various forms of malnutrition are expected to increase globally due to the current crises, though the exact impact on war zones is yet to be measured comprehensively.

In general, malnutrition in war zones can take several forms. Chronic malnutrition (also known as undernutrition) refers to deficiencies in energy and nutrient intake over a long period of time, resulting in slowed growth and development. Acute malnutrition occurs when there is an immediate lack of food or the body's nutritional requirements are not met due to illness, displacement, or conflict. Both forms can have severe health consequences, including increased susceptibility to disease, stunted physical and cognitive development, and in some cases, death.

Additionally, the context mentions the double burden of malnutrition, which refers to the co-existence of undernutrition (undernourishment, micronutrient deficien

In [125]:
res = query_rag("Explain increase prices impact on food security")
print(res)

 The context provided discusses the impact of increasing food prices on food security. Higher food prices can have a detrimental effect on food security, particularly for vulnerable households in certain regions. This is because a healthy diet becomes less affordable with rising prices, placing an additional burden on these households.

In 2022, the average cost of a healthy diet globally increased to 3.96 purchasing power parity (PPP) dollars per person per day, up from 3.56 PPP dollars in 2021. This price hike was caused by various factors such as disruptions from the COVID-19 pandemic and the war in Ukraine, which led to significant increases in international food and energy prices, thereby exacerbating inflationary pressures.

Moreover, countries that are heavily dependent on food imports were particularly affected by this rise in international food prices. The world food import bill reached an all-time high of nearly USD 2 trillion in 2022, resulting from the increased cost of foo

In [126]:
res = query_rag("Compare list of Food insecurity reason in 2023 and 2024")
print(res)

 Based on the provided context, there is no direct comparison between the reasons for food insecurity in 2023 and 2024 as the report only provides information up to 2023. However, it mentions that global hunger and food insecurity remained at high levels in 2023, with the following factors contributing: climatic, economic, social, and political instability. These factors can lead to acute or chronic food insecurity. The report does not explicitly mention any changes in these factors from 2023 to 2024. Therefore, without further information or a new report for 2024, it is impossible to compare the reasons for food insecurity between these two years.


In [127]:
res = query_rag("Percentage increase in global hunger")
print(res)

 According to the provided context, it is estimated that between 2019 and 2022, global hunger increased by approximately 122 million people, which represents an increase of around 15.3% (from 7.9% in 2019 to 9.2% in 2022). However, it's important to note that the data for 2023 is not provided in the context, so the percentage increase from 2022 to 2023 cannot be determined from this information.


In [128]:
res = query_rag("Percentage prevalence of undernourishment, Low Birthweight, and stunting.")
print(res)

1. Under-nourishment (Global): The percentage prevalence of undernourishment varies across regions, but according to World Bank data for 2019, the global average was approximately 8.4%. This means that 8.4% of the world's population is undernourished.

  2. Low Birthweight (Global): The percentage of low birthweight newborns worldwide in 2019 was about 14.6%. This indicates that, on average, for every 100 live births, 14.6 are classified as low birthweight.

  3. Stunting (Global): The global prevalence of stunting among children under the age of five in 2019 was approximately 22%. This means that about one-fifth (20%) of children under five years old are stunted, which can result from chronic malnutrition during their first 1000 days of life.

These percentages provide a snapshot of the prevalence of these conditions on a global scale and should be used as a reference for understanding their impact. However, it's important to note that these numbers can vary significantly within regio

In [129]:
Submission_data

{'Explain increase prices impact on food security - comparing 2023 and 2024 reports': {'Response': " The increases in global food prices throughout 2022 (as reported in the 2023 report) have significantly affected food security, as highlighted in the 2024 report. Here's a comparison between the two reports:\n\n    - In the 2023 report, the average cost of a healthy diet globally was 3.56 PPP dollars per person per day. However, this figure rose to 3.96 PPP dollars in 2022 (as reported in the 2024 report), marking an increase of 11% between 2021 and 2022. This surge represents a strain on household budgets, particularly for those already struggling with food insecurity.\n    - The World Food Import Bill was estimated to have reached an all-time high in 2022 (nearly USD $2 trillion), a 10% increase from the 2021 level. This significant rise in import costs affected countries heavily dependent on food imports, exacerbating their food security challenges.\n    - World fertilizer prices als

In [130]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [131]:
import pandas as pd

In [132]:
# Convert the nested dictionary to a DataFrame
Submission_df = pd.DataFrame.from_dict(Submission_data, orient='index')

# Reset the index and rename it to "query"
Submission_df.reset_index(inplace=True)
Submission_df.rename(columns={'index': 'query'}, inplace=True)

In [133]:
Submission_df

,query,Response,context_texts,Sources
0,Explain increase prices impact on food securit...,The increases in global food prices throughou...,cost and more accurate estimates of the afford...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...
1,List major Food insecurity reason in 2024,1. Climatic factors: These can include extreme...,lead to acute food insecurity) or medium- \nto...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...
2,Explain malnutrition in war zones,The provided context does not explicitly ment...,pathways for the pandemic to impact child \nnu...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...
3,Explain increase prices impact on food security,The context provided discusses the impact of ...,increases in the opportunity cost of time rela...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...
4,Compare list of Food insecurity reason in 2023...,"Based on the provided context, there is no di...",lead to acute food insecurity) or medium- \nto...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...
5,Percentage increase in global hunger,"According to the provided context, it is esti...",FOOD SECURITY AND NUTRITION IN THE WORLD\nTHE...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...
6,"Percentage prevalence of undernourishment, Low...",1. Under-nourishment (Global): The percentage ...,REGIONS/\nSUBREGIONS/\nCOUNTRIES/\nTERRITORIES...,[[/Users/rushabhrunwal/Downloads/SOFI_pdfs/SOF...


In [134]:
# Save the DataFrame to a CSV file
Submission_df.to_csv('Submission_data.csv', index=False)